In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, get_feature_names

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv', encoding = "ISO-8859-1")

In [3]:
movie_ratings = pd.merge(ratings, movies, on='movieId')

In [4]:
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,851527569,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,849082742,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,835562174,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,846509384,Jumanji (1995),Adventure|Children|Fantasy


In [5]:
sparse_features = ['userId', 'movieId', 'timestamp', 'title', 'genres']
target = ['rating']

In [6]:
for feature in sparse_features:
    lbe = LabelEncoder()
    movie_ratings[feature] = lbe.fit_transform(movie_ratings[feature])

In [7]:
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,0,1,3.5,340880,6746,440
1,4,1,3.0,39435,6746,440
2,12,1,3.0,35834,6746,440
3,28,1,3.0,11193,6746,440
4,33,1,3.0,31468,6746,440


In [8]:
fixlen_feature_columns = [SparseFeat(feature, movie_ratings[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [9]:
train, test = train_test_split(movie_ratings, test_size = 0.2)
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 
model = DeepFM(linear_feature_columns, dnn_feature_columns, task = 'regression')
model.compile('adam', 'mse', metrics=['mse'])
history = model.fit(train_model_input, train[target].values, batch_size=64, epochs = 10, verbose = True, validation_split=0.2)

Train on 671088 samples, validate on 167772 samples
Epoch 1/10


C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


671088/671088 [==============================] - 222s 331us/sample - loss: 0.8306 - mse: 0.8212 - val_loss: 0.7505 - val_mse: 0.7358
Epoch 2/10
671088/671088 [==============================] - 220s 327us/sample - loss: 0.6212 - mse: 0.5993 - val_loss: 0.7562 - val_mse: 0.7268
Epoch 3/10
671088/671088 [==============================] - 220s 327us/sample - loss: 0.5263 - mse: 0.4959 - val_loss: 0.7678 - val_mse: 0.7371
Epoch 4/10
671088/671088 [==============================] - 221s 329us/sample - loss: 0.4922 - mse: 0.4615 - val_loss: 0.7685 - val_mse: 0.7362
Epoch 5/10
671088/671088 [==============================] - 222s 330us/sample - loss: 0.4728 - mse: 0.4401 - val_loss: 0.7671 - val_mse: 0.7331
Epoch 6/10
671088/671088 [==============================] - 232s 346us/sample - loss: 0.4522 - mse: 0.4181 - val_loss: 0.7676 - val_mse: 0.7324
Epoch 7/10
671088/671088 [==============================] - 228s 340us/sample - loss: 0.4480 - mse: 0.4131 - val_loss: 0.7718 - val_mse: 0.7360
Epo

In [ ]:
test_model_input

In [ ]:
pred_ans = model.predict(test_model_input, batch_size = 64)

In [ ]:
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print('test RMSE', rmse)